In [1]:
import pandas as pd
import numpy as np

In [2]:
%store -r train

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train_data, test_data = train_test_split(train, test_size = 0.3)
train_data

,Country.Name,BAR.NOED.1519.FE.ZS,BAR.NOED.1519.ZS,BAR.NOED.15UP.FE.ZS,BAR.NOED.15UP.ZS,BAR.NOED.2024.FE.ZS,BAR.NOED.2024.ZS,BAR.NOED.2529.FE.ZS,BAR.NOED.2529.ZS,BAR.NOED.25UP.FE.ZS,...,UIS.XUNIT.US.56.FSGOV,UIS.XUNIT.USCONST.1.FSGOV,UIS.XUNIT.USCONST.2.FSGOV,UIS.XUNIT.USCONST.23.FSGOV,UIS.XUNIT.USCONST.3.FSGOV,UIS.XUNIT.USCONST.4.FSGOV,UIS.XUNIT.USCONST.56.FSGOV,XGDP.23.FSGOV.FDINSTADM.FFD,XGDP.56.FSGOV.FDINSTADM.FFD,Mean.Rank
44,Israel,34.0,35.0,55.0,57.0,59.0,63.0,58.0,60.0,55.0,...,24.0,21.0,73.0,27.0,11.0,25.0,23.0,45.0,43.0,29.00
7,Bangladesh,10.0,8.0,6.0,5.0,6.0,3.0,5.0,4.0,6.0,...,82.0,79.0,73.0,79.0,76.0,44.0,82.0,75.0,83.0,50.38
49,Kosovo,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,84.0,81.0,73.0,82.0,76.0,45.0,83.0,84.0,1.0,85.00
67,Pakistan,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,64.0,75.0,73.0,76.0,76.0,45.0,66.0,84.0,1.0,81.63
73,Portugal,68.0,71.0,27.0,29.0,64.0,64.0,56.0,54.0,27.0,...,25.0,26.0,22.0,24.0,24.0,45.0,25.0,19.0,45.0,44.88
54,Malaysia,40.0,37.0,30.0,32.0,42.0,39.0,38.0,39.0,28.0,...,27.0,44.0,40.0,44.0,42.0,28.0,24.0,23.0,3.0,41.75
45,Italy,70.0,75.0,37.0,43.0,65.0,70.0,60.0,63.0,38.0,...,22.0,13.0,17.0,18.0,20.0,2.0,21.0,27.0,64.0,22.00
76,Russia,72.0,73.0,74.0,76.0,71.0,71.0,70.0,69.0,73.0,...,65.0,81.0,73.0,82.0,76.0,45.0,56.0,84.0,60.0,5.50
24,Czech Republic,81.0,83.0,88.0,87.0,77.0,79.0,77.0,80.0,87.0,...,28.0,34.0,28.0,31.0,29.0,27.0,28.0,35.0,46.0,37.38
39,India,9.0,12.0,4.0,4.0,3.0,6.0,4.0,5.0,4.0,...,71.0,76.0,70.0,75.0,68.0,40.0,71.0,54.0,41.0,32.63


In [5]:
import xgboost as xgb

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train_data=train_data.fillna(np.nan)

In [8]:
%store -r train_columns
dtrain = xgb.DMatrix(data=train_data[train_columns],label=train_data['Mean.Rank'],missing=np.nan)
dtrain

In [9]:
param = {'objective': 'reg:linear', 'booster': 'gbtree', 'silent': 1,
         'max_depth': 10, 'eta': 0.1, 'nthread': 4,
         'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 20,
         'max_delta_step': 0, 'gamma': 0}
num_round = 1500


In [10]:
clf = xgb.train(param, dtrain, num_round)
dtest = xgb.DMatrix(test_data[train_columns], missing=np.nan)
test_data[ 'predicted_rank'] = clf.predict(dtest)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_data[train_columns])
scaler.transform(train_data[train_columns])

array([[ 0.37078652,  0.38202247,  0.60674157, ...,  0.25925926,
         0.53012048,  0.49411765],
       [ 0.1011236 ,  0.07865169,  0.05617978, ...,  0.98765432,
         0.89156627,  0.96470588],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  0.        ],
       ..., 
       [ 0.70786517,  0.71910112,  1.        , ...,  0.22222222,
         0.6626506 ,  0.81176471],
       [ 0.3258427 ,  0.30337079,  0.56179775, ...,  0.35802469,
         1.        ,  0.62352941],
       [ 0.56179775,  0.37078652,  0.34831461, ...,  0.43209877,
         0.45783133,  0.72941176]])

In [12]:
dtrain2 = xgb.DMatrix(data=scaler.transform(train_data[train_columns]),label=train_data['Mean.Rank'])
clf2 = xgb.train(param, dtrain2, num_round)
dtest2 = xgb.DMatrix(scaler.transform(test_data[train_columns]))
test_data[ 'predicted_rank2'] = clf2.predict(dtest2)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [13]:
test_data[['Mean.Rank','predicted_rank','predicted_rank2']]

,Mean.Rank,predicted_rank,predicted_rank2
41,12.63,45.801247,45.887974
85,57.75,48.182293,47.622589
58,92.25,52.034267,53.192669
55,26.75,42.650089,45.608257
89,10.00,43.433231,45.234085
9,51.75,56.669090,50.881744
90,86.38,85.697090,82.073723
87,53.88,57.697620,59.759602
57,44.88,59.152821,60.967461
4,19.75,30.730347,30.592190


In [14]:
test_data['Mean.Rank'].corr(test_data['predicted_rank2'])

0.73923350744753369

In [15]:
test_data['Mean.Rank'].corr(test_data['predicted_rank'])

0.74940261347285286